# **Image Classification: US License Plates**
## **Background information:**
This project consists of a very high-quality dataset, US license plate images from 50 states i.e., total 50 classes for classification. Based on the information provided, all images are originals, no augmented images are present in the dataset. All images are of size 128 pixels X 224 pixels X 3 channels in jpg format. All images have been cropped so the license plate occupies at least 90% of the pixels in any image. This ensures that even simple models will achieve high training, validation and test accuracy. Also included is a csv file that can be used to create train, validation and test sets if desired.

## **Dataset source:**
https://www.kaggle.com/datasets/gpiosenka/us-license-plates-image-classification

## **Criteria for success:**
Delivering a model with > 90% accuracy to classify a license plate image into one of different available 50 classes.






# **Modeling Approach I. Non-Neural Network Classifier**

This section would examine the performance of following two non-neural network classifiers

1.   Random Forest Classfier
2.   K-Nearest Neighbors (KNN) Classifier



## **Model A: Random Forest Classifier**

In [ ]:
# Import relevant python libraries
import cv2
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelBinarizer


# Function to load and preprocess images
def load_and_preprocess_images(data_dir):
    images = []
    labels = []

    for state_folder in os.listdir(data_dir):
        state_path = os.path.join(data_dir, state_folder)
        if os.path.isdir(state_path):
            for filename in os.listdir(state_path):
                img_path = os.path.join(state_path, filename)
                img = cv2.imread(img_path)
                images.append(img.flatten())  # Flatten the image array
                labels.append(state_folder)

    return np.array(images), np.array(labels)

# Load and preprocess data
data_dir = "drive/MyDrive/DSC Capstone 3/plates/train"
images, labels = load_and_preprocess_images(data_dir)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Encode labels
label_binarizer = LabelBinarizer()
y_train_encoded = label_binarizer.fit_transform(y_train)
y_test_encoded = label_binarizer.transform(y_test)

# Create and train the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train_encoded)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test_encoded, y_pred)
print(f"Accuracy: {accuracy*100:.2f}", "%")

Accuracy: 3.19 %


## **Model B: K-Nearest Neighbors (KNN) Classifier**

In [ ]:
# Create and train the KNN Classifier
knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(X_train, y_train_encoded)

# Make predictions on the test set
y_pred = knn_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test_encoded, y_pred)
print(f"Accuracy: {accuracy*100:.2f}", "%")


Accuracy: 16.64 %


# **Modeling Approach II. Neural Networks (Deep Learning) Classifiers**

This section would examine the performance of following three neural network *aka* deep learning classifiers

1.   Multilevel Perceptron (MLP) Classfier
2.   Convolutional Neural Network (CNN) Classifier
3.   Transfer Learning (ResNet Model) Classifier


## **Model A: Multilevel Perceptron (MLP) Classfier**

In [ ]:
# Convert to TensorFlow tensors and normalize
X_train_tensor = tf.convert_to_tensor(X_train / 255.0, dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train_encoded, dtype=tf.float32)

# Build the MLP model using TensorFlow and Keras
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(label_binarizer.classes_), activation='softmax')
])

# Compile the model
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

# Convert tensors to a tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_tensor, y_train_tensor))

# Train the model using the tf.data.Dataset with batch size 32
model.fit(train_dataset.batch(32), epochs=10)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test / 255.0, y_test_encoded, verbose=2)
print('\n'f"Test Accuracy: {test_accuracy*100:.2f}", "%")

Epoch 1/10
184/184 [==============================] - 2s 8ms/step - loss: 4.0034 - accuracy: 0.0222
Epoch 2/10
184/184 [==============================] - 1s 8ms/step - loss: 3.9141 - accuracy: 0.0248
Epoch 3/10
184/184 [==============================] - 2s 8ms/step - loss: 3.9117 - accuracy: 0.0248
Epoch 4/10
184/184 [==============================] - 2s 8ms/step - loss: 3.9115 - accuracy: 0.0250
Epoch 5/10
184/184 [==============================] - 1s 8ms/step - loss: 3.9113 - accuracy: 0.0250
Epoch 6/10
184/184 [==============================] - 1s 8ms/step - loss: 3.9111 - accuracy: 0.0250
Epoch 7/10
184/184 [==============================] - 1s 8ms/step - loss: 3.9110 - accuracy: 0.0250
Epoch 8/10
184/184 [==============================] - 1s 8ms/step - loss: 3.9147 - accuracy: 0.0251
Epoch 9/10
184/184 [==============================] - 1s 8ms/step - loss: 3.9107 - accuracy: 0.0250
Epoch 10/10
184/184 [==============================] - 1s 8ms/step - loss: 3.9106 - accuracy: 0.0250

### **MLP Model: Increasing number of hidden layers in the model (Deep neural network)**

In [ ]:
# Build the MLP model using TensorFlow and Keras (5 hidden layers)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(50, activation='softmax')
])

# Compile the model
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

# Convert tensors to a tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_tensor, y_train_tensor))


# Train the model using the tf.data.Dataset with batch size 32
model.fit(train_dataset.batch(32), epochs=10)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test / 255.0, y_test_encoded, verbose=2)
print('\n'f"Test Accuracy: {test_accuracy*100:.2f}", "%")

Epoch 1/10
184/184 [==============================] - 3s 14ms/step - loss: 3.9148 - accuracy: 0.0229
Epoch 2/10
184/184 [==============================] - 3s 14ms/step - loss: 3.8897 - accuracy: 0.0297
Epoch 3/10
184/184 [==============================] - 3s 14ms/step - loss: 3.8559 - accuracy: 0.0403
Epoch 4/10
184/184 [==============================] - 3s 14ms/step - loss: 3.8156 - accuracy: 0.0521
Epoch 5/10
184/184 [==============================] - 3s 14ms/step - loss: 3.7591 - accuracy: 0.0749
Epoch 6/10
184/184 [==============================] - 3s 14ms/step - loss: 3.6829 - accuracy: 0.0929
Epoch 7/10
184/184 [==============================] - 3s 14ms/step - loss: 3.6009 - accuracy: 0.1104
Epoch 8/10
184/184 [==============================] - 3s 14ms/step - loss: 3.5103 - accuracy: 0.1398
Epoch 9/10
184/184 [==============================] - 3s 14ms/step - loss: 3.4159 - accuracy: 0.1566
Epoch 10/10
184/184 [==============================] - 3s 14ms/step - loss: 3.3145 - accura

In [ ]:
# Build the MLP model using TensorFlow and Keras (4 hidden layers)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(50, activation='softmax')
])

# Compile the model
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

# Convert tensors to a tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_tensor, y_train_tensor))


# Train the model using the tf.data.Dataset with batch size 32
model.fit(train_dataset.batch(32), epochs=10)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test / 255.0, y_test_encoded, verbose=2)
print('\n'f"Test Accuracy: {test_accuracy*100:.2f}", "%")

Epoch 1/10
184/184 [==============================] - 3s 10ms/step - loss: 3.9155 - accuracy: 0.0224
Epoch 2/10
184/184 [==============================] - 2s 10ms/step - loss: 3.8902 - accuracy: 0.0263
Epoch 3/10
184/184 [==============================] - 2s 11ms/step - loss: 3.8638 - accuracy: 0.0362
Epoch 4/10
184/184 [==============================] - 2s 10ms/step - loss: 3.8217 - accuracy: 0.0476
Epoch 5/10
184/184 [==============================] - 2s 10ms/step - loss: 3.7708 - accuracy: 0.0667
Epoch 6/10
184/184 [==============================] - 2s 10ms/step - loss: 3.7075 - accuracy: 0.0851
Epoch 7/10
184/184 [==============================] - 2s 10ms/step - loss: 3.6256 - accuracy: 0.1072
Epoch 8/10
184/184 [==============================] - 2s 10ms/step - loss: 3.5455 - accuracy: 0.1277
Epoch 9/10
184/184 [==============================] - 2s 10ms/step - loss: 3.4354 - accuracy: 0.1501
Epoch 10/10
184/184 [==============================] - 2s 10ms/step - loss: 3.3414 - accura

In [ ]:
# Build the MLP model using TensorFlow and Keras (3 hidden layers)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(50, activation='softmax')
])

# Compile the model
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

# Convert tensors to a tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_tensor, y_train_tensor))


# Train the model using the tf.data.Dataset with batch size 32
model.fit(train_dataset.batch(32), epochs=10)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test / 255.0, y_test_encoded, verbose=2)
print('\n'f"Test Accuracy: {test_accuracy*100:.2f}", "%")

Epoch 1/10
184/184 [==============================] - 2s 8ms/step - loss: 3.9263 - accuracy: 0.0245
Epoch 2/10
184/184 [==============================] - 1s 8ms/step - loss: 3.8933 - accuracy: 0.0290
Epoch 3/10
184/184 [==============================] - 2s 8ms/step - loss: 3.8753 - accuracy: 0.0358
Epoch 4/10
184/184 [==============================] - 2s 8ms/step - loss: 3.8572 - accuracy: 0.0348
Epoch 5/10
184/184 [==============================] - 2s 8ms/step - loss: 3.8374 - accuracy: 0.0370
Epoch 6/10
184/184 [==============================] - 1s 8ms/step - loss: 3.8168 - accuracy: 0.0382
Epoch 7/10
184/184 [==============================] - 2s 8ms/step - loss: 3.7967 - accuracy: 0.0399
Epoch 8/10
184/184 [==============================] - 2s 8ms/step - loss: 3.7683 - accuracy: 0.0479
Epoch 9/10
184/184 [==============================] - 1s 8ms/step - loss: 3.7449 - accuracy: 0.0566
Epoch 10/10
184/184 [==============================] - 1s 8ms/step - loss: 3.7098 - accuracy: 0.0673

In [ ]:
# Build the MLP model using TensorFlow and Keras (2 hidden layers)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(50, activation='softmax')
])

# Compile the model
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

# Convert tensors to a tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_tensor, y_train_tensor))


# Train the model using the tf.data.Dataset with batch size 32
model.fit(train_dataset.batch(32), epochs=10)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test / 255.0, y_test_encoded, verbose=2)
print('\n'f"Test Accuracy: {test_accuracy*100:.2f}", "%")

Epoch 1/10
184/184 [==============================] - 2s 7ms/step - loss: 3.9650 - accuracy: 0.0214
Epoch 2/10
184/184 [==============================] - 1s 7ms/step - loss: 3.9133 - accuracy: 0.0255
Epoch 3/10
184/184 [==============================] - 1s 7ms/step - loss: 3.9116 - accuracy: 0.0251
Epoch 4/10
184/184 [==============================] - 1s 7ms/step - loss: 3.9113 - accuracy: 0.0251
Epoch 5/10
184/184 [==============================] - 1s 7ms/step - loss: 3.9122 - accuracy: 0.0239
Epoch 6/10
184/184 [==============================] - 1s 7ms/step - loss: 3.9111 - accuracy: 0.0248
Epoch 7/10
184/184 [==============================] - 1s 7ms/step - loss: 3.9109 - accuracy: 0.0245
Epoch 8/10
184/184 [==============================] - 1s 7ms/step - loss: 3.9111 - accuracy: 0.0251
Epoch 9/10
184/184 [==============================] - 1s 7ms/step - loss: 3.9109 - accuracy: 0.0250
Epoch 10/10
184/184 [==============================] - 1s 7ms/step - loss: 3.9103 - accuracy: 0.0250

### **MLP Model: Increasing epochs**

This section studies the model perfomance with increasing number of epochs

In [ ]:
# Build the MLP model using TensorFlow and Keras (4 hidden layers)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(50, activation='softmax')
])

# Compile the model
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

# Convert tensors to a tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_tensor, y_train_tensor))


# Train the model using the tf.data.Dataset with batch size 32
model.fit(train_dataset.batch(32), epochs=20) #2x epochs

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test / 255.0, y_test_encoded, verbose=2)
print('\n'f"Test Accuracy: {test_accuracy*100:.2f}", "%")

Epoch 1/20
184/184 [==============================] - 3s 10ms/step - loss: 3.9196 - accuracy: 0.0217
Epoch 2/20
184/184 [==============================] - 2s 10ms/step - loss: 3.9008 - accuracy: 0.0228
Epoch 3/20
184/184 [==============================] - 2s 10ms/step - loss: 3.8715 - accuracy: 0.0331
Epoch 4/20
184/184 [==============================] - 2s 10ms/step - loss: 3.8385 - accuracy: 0.0494
Epoch 5/20
184/184 [==============================] - 2s 10ms/step - loss: 3.7921 - accuracy: 0.0623
Epoch 6/20
184/184 [==============================] - 2s 10ms/step - loss: 3.7296 - accuracy: 0.0730
Epoch 7/20
184/184 [==============================] - 2s 10ms/step - loss: 3.6587 - accuracy: 0.0953
Epoch 8/20
184/184 [==============================] - 2s 10ms/step - loss: 3.5877 - accuracy: 0.1170
Epoch 9/20
184/184 [==============================] - 2s 10ms/step - loss: 3.4829 - accuracy: 0.1413
Epoch 10/20
184/184 [==============================] - 2s 10ms/step - loss: 3.3959 - accura

In [ ]:
# Build the MLP model using TensorFlow and Keras (4 hidden layers)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(50, activation='softmax')
])

# Compile the model
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

# Convert tensors to a tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_tensor, y_train_tensor))


# Train the model using the tf.data.Dataset with batch size 32
model.fit(train_dataset.batch(32), epochs=30) # 3x epochs

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test / 255.0, y_test_encoded, verbose=2)
print('\n'f"Test Accuracy: {test_accuracy*100:.2f}", "%")

Epoch 1/30
184/184 [==============================] - 3s 10ms/step - loss: 3.9211 - accuracy: 0.0197
Epoch 2/30
184/184 [==============================] - 2s 10ms/step - loss: 3.8986 - accuracy: 0.0279
Epoch 3/30
184/184 [==============================] - 2s 10ms/step - loss: 3.8767 - accuracy: 0.0369
Epoch 4/30
184/184 [==============================] - 2s 10ms/step - loss: 3.8413 - accuracy: 0.0496
Epoch 5/30
184/184 [==============================] - 2s 10ms/step - loss: 3.7913 - accuracy: 0.0596
Epoch 6/30
184/184 [==============================] - 2s 11ms/step - loss: 3.7271 - accuracy: 0.0764
Epoch 7/30
184/184 [==============================] - 2s 10ms/step - loss: 3.6565 - accuracy: 0.0875
Epoch 8/30
184/184 [==============================] - 2s 10ms/step - loss: 3.5831 - accuracy: 0.1028
Epoch 9/30
184/184 [==============================] - 2s 10ms/step - loss: 3.4999 - accuracy: 0.1194
Epoch 10/30
184/184 [==============================] - 2s 10ms/step - loss: 3.4172 - accura

### **MLP Model: Increasing number of neurons in each hidden layer**

In [ ]:
# Build the MLP model using TensorFlow and Keras (4 hidden layers)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])),
    tf.keras.layers.Dense(512, activation='relu'), #2x number of neurons
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(50, activation='softmax')
])

# Compile the model
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

# Convert tensors to a tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_tensor, y_train_tensor))


# Train the model using the tf.data.Dataset with batch size 32
model.fit(train_dataset.batch(32), epochs=20)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test / 255.0, y_test_encoded, verbose=2)
print('\n'f"Test Accuracy: {test_accuracy*100:.2f}", "%")

Epoch 1/20
184/184 [==============================] - 3s 14ms/step - loss: 3.9196 - accuracy: 0.0226
Epoch 2/20
184/184 [==============================] - 3s 14ms/step - loss: 3.8701 - accuracy: 0.0409
Epoch 3/20
184/184 [==============================] - 3s 14ms/step - loss: 3.7896 - accuracy: 0.0594
Epoch 4/20
184/184 [==============================] - 3s 14ms/step - loss: 3.6823 - accuracy: 0.0885
Epoch 5/20
184/184 [==============================] - 3s 14ms/step - loss: 3.5570 - accuracy: 0.1213
Epoch 6/20
184/184 [==============================] - 3s 14ms/step - loss: 3.4128 - accuracy: 0.1602
Epoch 7/20
184/184 [==============================] - 3s 14ms/step - loss: 3.2714 - accuracy: 0.1909
Epoch 8/20
184/184 [==============================] - 3s 14ms/step - loss: 3.1262 - accuracy: 0.2244
Epoch 9/20
184/184 [==============================] - 3s 14ms/step - loss: 3.0194 - accuracy: 0.2556
Epoch 10/20
184/184 [==============================] - 3s 14ms/step - loss: 2.8917 - accura

**Key takeaways:**

1.   MLP classifier with 4 hidden layers improved test data accuracy by more than an order of magnitude, i.e., from a test data accuracy of 1.9% -> 13.8%
2.   MLP classifier with 2x epochs further improve the accuracy of the model by 2x, i.e. from a test data accuracy of 13.8 -> 27.8%



## **Model B: Convolutional Neural Network (CNN) Classifier**

In [ ]:
# Build the CNN model using TensorFlow and Keras
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(label_binarizer.classes_), activation='softmax')
])

# Compile the model
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

# Convert tensors to a tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train_encoded))

# Train the model using the tf.data.Dataset with batch size 32
model.fit(train_dataset.batch(32), epochs=10)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test_encoded, verbose=2)
print('\n'f"Test Accuracy: {test_accuracy*100:.2f}", "%")


Epoch 1/10
184/184 [==============================] - 6s 28ms/step - loss: 3.9141 - accuracy: 0.0229
Epoch 2/10
184/184 [==============================] - 5s 28ms/step - loss: 3.8892 - accuracy: 0.0282
Epoch 3/10
184/184 [==============================] - 5s 28ms/step - loss: 3.7909 - accuracy: 0.0679
Epoch 4/10
184/184 [==============================] - 5s 28ms/step - loss: 3.4982 - accuracy: 0.1680
Epoch 5/10
184/184 [==============================] - 5s 28ms/step - loss: 2.9286 - accuracy: 0.3127
Epoch 6/10
184/184 [==============================] - 5s 28ms/step - loss: 2.2565 - accuracy: 0.4621
Epoch 7/10
184/184 [==============================] - 5s 28ms/step - loss: 1.6220 - accuracy: 0.6058
Epoch 8/10
184/184 [==============================] - 5s 28ms/step - loss: 1.0142 - accuracy: 0.7537
Epoch 9/10
184/184 [==============================] - 5s 28ms/step - loss: 0.5243 - accuracy: 0.8792
Epoch 10/10
184/184 [==============================] - 5s 28ms/step - loss: 0.3173 - accura

### **CNN Model: Implementing batch normalization**

In [ ]:
# Build the CNN model using TensorFlow and Keras with Batch Normalization
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]), padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(64, (3, 3), padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(128),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),

    tf.keras.layers.Dense(len(label_binarizer.classes_), activation='softmax')
])

# Compile the model
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

# Convert tensors to a tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train_encoded))

# Train the model using the tf.data.Dataset with batch size 32
model.fit(train_dataset.batch(32), epochs=10)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test_encoded, verbose=2)
print('\n'f"Test Accuracy: {test_accuracy*100:.2f}", "%")


Epoch 1/10
184/184 [==============================] - 11s 39ms/step - loss: 2.6814 - accuracy: 0.4110
Epoch 2/10
184/184 [==============================] - 7s 39ms/step - loss: 1.1841 - accuracy: 0.8485
Epoch 3/10
184/184 [==============================] - 7s 39ms/step - loss: 0.4898 - accuracy: 0.9832
Epoch 4/10
184/184 [==============================] - 7s 39ms/step - loss: 0.2078 - accuracy: 0.9985
Epoch 5/10
184/184 [==============================] - 7s 39ms/step - loss: 0.1150 - accuracy: 0.9993
Epoch 6/10
184/184 [==============================] - 7s 40ms/step - loss: 0.0793 - accuracy: 0.9998
Epoch 7/10
184/184 [==============================] - 7s 40ms/step - loss: 0.0610 - accuracy: 0.9997
Epoch 8/10
184/184 [==============================] - 7s 40ms/step - loss: 0.0495 - accuracy: 0.9998
Epoch 9/10
184/184 [==============================] - 7s 40ms/step - loss: 0.0417 - accuracy: 0.9998
Epoch 10/10
184/184 [==============================] - 7s 39ms/step - loss: 0.0360 - accur

**Key takeaways:**

1.   CNN model with 32 filters and max pooling 2D showed 57.3% test accuracy
2.   CNN model with data augmentation further improved the accuracy by 15-20%,  i.e., from a test data accuracy of 57.6 -> 67.6%


## **Model C: Transfer Learning Implementation**

In [ ]:
# Load pre-trained ResNet50 model (with top layers)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Create a custom model
model = Sequential([
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(50, activation='softmax')
])

# Compile the model
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

# Convert tensors to a tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train_encoded))

# Train the model using the tf.data.Dataset with batch size 32
model.fit(train_dataset.batch(32), epochs=10)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test_encoded, verbose=2)
print('\n'f"Test Accuracy: {test_accuracy*100:.2f}", "%")

94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/10
184/184 [==============================] - 82s 338ms/step - loss: 3.7416 - accuracy: 0.4400
Epoch 2/10
184/184 [==============================] - 61s 334ms/step - loss: 0.5680 - accuracy: 0.8718
Epoch 3/10
184/184 [==============================] - 62s 335ms/step - loss: 0.0554 - accuracy: 0.9924
Epoch 4/10
184/184 [==============================] - 62s 334ms/step - loss: 0.0101 - accuracy: 0.9995
Epoch 5/10
184/184 [==============================] - 62s 334ms/step - loss: 0.0057 - accuracy: 0.9997
Epoch 6/10
184/184 [==============================] - 62s 334ms/step - loss: 0.0042 - accuracy: 0.9997
Epoch 7/10
184/184 [==============================] - 62s 334ms/step - loss: 0.0038 - accuracy: 0.9997
Epoch 8/10
184/184 [==============================] - 62s 334ms/step - loss: 0.0032 - accuracy: 0.9998
Epoch 9/10
184/184 [==============================] - 62s 334ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 10

### **Transfer Learning: Applying data augmentation**

In [1]:
# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Compile the model
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

# Convert tensors to a tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train_encoded))

# Apply data augmentation and train the model using the tf.data.Dataset with batch size 32
model.fit(datagen.flow(X_train, y_train_encoded, batch_size=32), epochs=20)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test_encoded, verbose=2)
print('\n'f"Test Accuracy: {test_accuracy*100:.2f}", "%")


94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/20
184/184 [==============================] - 82s 326ms/step - loss: 4.5971 - accuracy: 0.1731
Epoch 2/20
184/184 [==============================] - 64s 348ms/step - loss: 2.4255 - accuracy: 0.4129
Epoch 3/20
184/184 [==============================] - 64s 345ms/step - loss: 1.8082 - accuracy: 0.5510
Epoch 4/20
184/184 [==============================] - 64s 346ms/step - loss: 1.3889 - accuracy: 0.6423
Epoch 5/20
184/184 [==============================] - 64s 346ms/step - loss: 1.1050 - accuracy: 0.7091
Epoch 6/20
184/184 [==============================] - 64s 346ms/step - loss: 0.8888 - accuracy: 0.7627
Epoch 7/20
184/184 [==============================] - 64s 347ms/step - loss: 0.6859 - accuracy: 0.8098
Epoch 8/20
184/184 [==============================] - 64s 348ms/step - loss: 0.5947 - accuracy: 0.8402
Epoch 9/20
184/184 [==============================] - 64s 348ms/step - loss: 0.4739 - accuracy: 0.8691
Epoch 10

**Key takeaways:**

1.   Transfer learning ResNet pre-trained model showed an improvement in test data accuracy over CNN model by 15-20%
2.   Transfer learning with data augmentation further improved the test data accuracy by 10%, i.e. from 75.8 -> 83.1%
3.   Transfer learning showed the best test data accuracy ~ 85% close to our target ~ 90% 

